In [ ]:
%%capture
# Here, we add an environment variable to help theano locate the BLAS libraries.
# found in ./usr/lib/x86_64-linux-gnu/libblas.so on colab
# More info at http://deeplearning.net/software/theano/troubleshooting.html
import os
os.environ["THEANO_FLAGS"] = 'blas.ldflags="-L/usr/lib/x86_64-linux-gnu/openblas -lblas"' # Google Colab default

# Model
! pip install git+https://github.com/Priesemann-Group/covid19_inference.git
! pip install git+https://gitlab.com/stevenhorn/covid-prevalence-estimate.git

# Data
! git clone https://github.com/ishaberry/Covid19Canada.git --depth 1 --branch master --single-branch /content/Covid19Canada
! git clone https://github.com/CSSEGISandData/COVID-19.git --depth 1 --branch master --single-branch /content/COVID-19

# Configuration
! wget https://gitlab.com/stevenhorn/covid-prevalence-estimate/raw/master/config/config.json

In [ ]:
import datetime, json, urllib.request
import numpy as np
import pymc3 as pm
import pymc3.stats as pms
import matplotlib.pyplot as plt
from covid_prevalence.data import get_data, savecsv
from covid_prevalence.models import PrevModel
from covid_prevalence.utility import get_folders
from covid_prevalence.plots import plot_fit, plot_prevalence, plot_posteriors, plot_introduction

plt.style.use('seaborn-darkgrid')

In [ ]:
# Load configuration and population
with open('/content/config.json','r') as f:
  config = json.load(f)
  
model = config['settings']['model']
settings = config['settings']
settings['model'] = model
pops = config['populations']
# filter for a single population.
pops = [p for p in config['populations'] if p['source_region'] == 'Logan' and p['source_state'] == 'West Virginia']
pop = pops[0]

# render
new_cases, cum_deaths, bd = get_data(pops[0])  
new_cases[new_cases < 0] = 0
plt.plot(new_cases)
plt.xticks(rotation=45)
plt.xlabel("Day")
plt.ylabel("New cases");
plt.title(f"{pops[0]['name']} Health Region")

In [ ]:
# Run the model on the latest data
new_cases, cum_deaths, bd = get_data(pop)  # plt.plot(new_cases)

# fix up negative case values
new_cases[new_cases < 0] = 0

#pop['normal_likelihood'] = True

params_model = dict(
      new_cases_obs=new_cases,
      data_begin=bd,
      fcast_len=14,             # forecast model
      diff_data_sim=5,     # number of days for burn-in
      N_population=pop['N'],
      settings=settings,
      pop = pop,
    )

numsims = 20#settings['numsims']
numtune = 20#settings['numtune']

# Create the model
with PrevModel(**params_model) as this_model:

  # initialize and sample the model
  trace = pm.sample(model=this_model, 
                    tune=numtune, 
                    draws=numsims, 
                    n_init=50000, 
                    init="advi+adapt_diag", 
                    cores=2, 
                    target_accept=0.95)


In [ ]:
with this_model:
  savefolder, folder = get_folders(pop, rootpath='/content')

  plot_fit(this_model, trace, new_cases, pop, settings, closeplot=False, rootpath='/content')
  plot_prevalence(this_model, trace, pop, settings, closeplot=False, rootpath='/content')

  divs = trace.get_sampler_stats('diverging')
  pop['divs'] = np.sum(divs)
  llostat = pms.loo(trace,pointwise=True, scale="log")
  llostat_str = str(llostat)
  summary = pm.summary(trace, var_names=["pa", "pu","mu","mus", "gamma", "Is_begin","Ia_begin","E_begin"])
  summary_str = str(summary)
  savepath = savefolder + '/'+folder+'_stats.txt'
  with open(savepath, 'w') as f:
    f.write('%d Divergences \n' % np.sum(divs))
    f.write(llostat_str)
    f.write('\n')
    f.write(summary_str)

  pop['compute_time'] = datetime.datetime.utcnow() - datetime.datetime.utcnow()
  pop['draws'] = 20
  pop['tunes'] = 20

  plot_fit(this_model, trace, new_cases, pop, settings, closeplot=True, rootpath='/content/drive/My Drive/covid-prev')
  plot_prevalence(this_model, trace, pop, settings, closeplot=True, rootpath='/content/drive/My Drive/covid-prev')
  plot_introduction(this_model, trace, pop, settings, closeplot=False)
  _, _ = savecsv(this_model, trace, pop, rootpath='/content/drive/My Drive/covid-prev')

In [ ]:
with this_model:
  summary = pm.summary(trace, var_names=["pa", "pu","mu","mus", "gamma", "Is_begin","Ia_begin","E_begin"])
  print(summary)